In [ ]:
import sys
import os

project_root = os.path.abspath("..")  
sys.path.append(project_root)

import numpy as np
import random
from stable_baselines3 import PPO
import matplotlib.pyplot as plt

from src.env.TestEnv import HydroElectric_Test
from src.models.PPO import PPOWrapper

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [2]:
seed = 5
np.random.seed(seed)
random.seed(seed)

train_path = os.path.abspath("../data/train.xlsx")
env = HydroElectric_Test(train_path)
env = PPOWrapper(env, reward_shape=True)

model = PPO(
    "MlpPolicy",
    env,
    verbose=1,
    seed=seed
)

model.learn(total_timesteps=3_000,log_interval=100)
model.save("ppo_model")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [3]:
val_path = os.path.abspath("../data/validate.xlsx")
env_test = HydroElectric_Test(val_path)
env_test = PPOWrapper(env_test, reward_shape=False)

model = PPO.load("ppo_model")
obs, _ = env_test.reset()

done = False
water_levels = []
rewards = []
actions = []

while not done:
    action, _ = model.predict(obs, deterministic=True)
    
    obs, reward, terminated, truncated, _ = env_test.step(action)
    done = terminated or truncated
    
    water_levels.append(env.env.volume) 
    rewards.append(reward)
    actions.append(action[0])

print("Total reward:", round(sum(rewards),2))
print("Mean reward:", round(np.mean(rewards),3))


Total reward: -5905.07
Mean reward: -0.337


In [4]:
print(len(set(water_levels)))
print(len(set(actions)))
print(len(set(rewards)))

1
17513
17519
